# Imports

In [1]:
!pip install -q efficientnet_pytorch

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
from fastai.vision.all import *
from fastai.vision.core import *
from fastai.callback.fp16 import *

import pandas as pd
import numpy as np

from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import train_test_split, StratifiedKFold

import albumentations

import wandb
from fastai.callback.wandb import *

# Setup

In [3]:
path_str = '../input/cassava-leaf-disease-classification'

images_path = Path(path_str + '/train_images')
csv_path = Path(path_str + '/train.csv')
full_df = pd.read_csv(csv_path)

In [4]:
class Config:
    testing     = False
    image_size  = 512
    batch_size  = 16
    folds       = 3
    epochs      = 10
    f_epochs    = 1
    arch        = 'efficientnet-b4'
    
cfg = Config()

# Create a test dataset

In [5]:
if cfg.testing:
    msk = np.random.rand(len(full_df)) < 0.01
    full_df = full_df[msk]
else:
    wandb.login(key="11b470b697ff94b3896d2243b147d42177a5cb7a")
    wandb.init(project="cassava", entity="teo03")
len(full_df)

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: teo03 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.14 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


21397

# Augmentation and train functions

In [6]:
class AlbumentationsTransform(RandTransform):
    split_idx,order = None, 2
    
    def __init__(self, train_aug, valid_aug): 
        store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)


def get_train_aug(size): 
    return albumentations.Compose([
            #albumentations.Resize(size, size), # keep this?
            albumentations.RandomResizedCrop(size,size),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5)
])

def get_valid_aug(size): 
    return albumentations.Compose([
        albumentations.CenterCrop(size, size, p=1.),
        albumentations.Resize(size, size),
], p=1.)

In [7]:
def train(dls, fold_i):
    
    model = EfficientNet.from_pretrained(cfg.arch, num_classes=5)

    # define learner
    learn = Learner(
        dls=dls,
        model=model,
        opt_func=ranger,
        metrics=accuracy,
        loss_func=LabelSmoothingCrossEntropy(),
        cbs=[WandbCallback(log_preds=False, log_model=False, n_preds=10)],
    ).to_fp16()
    
    lr_min, lr_steep = learn.lr_find(show_plot=False)
    print(f'found lr of({lr_min}): {round(lr_min, 5)}')
    
    
    # start model training
    learn.fine_tune(
        cfg.epochs,
        base_lr=round(lr_min, 5),
        freeze_epochs=cfg.f_epochs
    )
    
    learn.export(Path(f'model-f{fold_i}.pkl'))
    
    return learn

In [8]:
def get_x(row): return images_path/row['image_id']
def get_y(row): return row['label']

# Training
Split to train/test sets and train model on different folds using k-fold

In [9]:
train_df, test_df = train_test_split(full_df, test_size=0.1)
len(train_df), len(test_df)

(19257, 2140)

In [10]:
skf = StratifiedKFold(n_splits=cfg.folds, shuffle=True)
i = 0

for train_index, val_index in skf.split(train_df.index, train_df['label']):

    print(f'training on fold: {i} \n' )
    
    train_block = DataBlock(
            blocks=(ImageBlock, CategoryBlock),
            get_x=get_x,
            get_y=get_y,
            splitter=IndexSplitter(val_index),
            item_tfms= [
                AlbumentationsTransform(
                    get_train_aug(size=cfg.image_size),
                    get_valid_aug(size=cfg.image_size)
                )
            ],
            batch_tfms=[Normalize.from_stats(*imagenet_stats)]
        )
    
    dls = train_block.dataloaders(train_df, bs=cfg.batch_size)
    learn = train(dls, i)
    
    print('done \n')
    
    i+=1

print('training done \n')

training on fold: 0 



Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b4-6ed6700e.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b4-6ed6700e.pth



Loaded pretrained weights for efficientnet-b4


found lr of(0.0006309573538601399): 0.00063


epoch,train_loss,valid_loss,accuracy,time
0,0.772769,0.802121,0.798567,14:42


...

done 

training on fold: 2 

Loaded pretrained weights for efficientnet-b4


found lr of(0.0019054606556892395): 0.00191


epoch,train_loss,valid_loss,accuracy,time
0,0.878635,0.962490,0.762424,14:49


epoch,train_loss,valid_loss,accuracy,time
0,0.718000,0.663387,0.871320,14:50
1,0.733389,0.665876,0.873656,14:50
2,0.731055,0.692821,0.862284,14:52
3,0.713185,0.692696,0.858545,14:49
4,0.686808,0.675973,0.873189,15:12
5,0.672478,0.642075,0.880823,15:17
6,0.643822,0.642767,0.883938,15:26
7,0.627007,0.637180,0.881913,15:09
8,0.627621,0.621103,0.889079,15:25
9,0.608641,0.621857,0.890326,15:31


done 

training done 

